# Reward Program
### Test from Jerry

#### Answer by Ellen Wang, <EllenwAnalyst@outlook.com>

### 1) 'What' information would you derive from it, and 'how' will you derive it 
(you can give SQL queries, pseudo code, ... whatever you're comfortable with)

#### Goals and Questions
**Goal:**

To evaluate if running this program has been a good idea or not

**Problems to ask:**

* What's the impact of this strategy: How many (new vs. recurring) users have participanted in the reward program, got the 10 dollar discount; how many of these users have redeemed the 10 dollar discount?
* Have the number of new customers increased after the reward program begins? How many new users get the 20 dollar discount? (actual impact of this promo event)
* Does revenue (sum of total pays) increase after implementing this strategy?

**Assumptions:**

* Consumers can only use either 10 or 20 dollar at one time.(Only users purchased at least once can get access to the 10 dollar discount)
* Users who have purchased and redeemed 20 dollar can not get 20 dollar discount again
* Assume the campaign started from March 1st
* No obvious business cycle (market for Jan-May is similar to market for Mar-May), if not, comaring with the corresponding period of last year.

**Note: Questions to ask internal stakeholders**

* Can users redeem these two discounts in the same order?


#### SQL CODE

In [17]:
sql='''
--Goal1: To examine the difference between pre- and post- program in terms of
--       total number of existing users that participant in the program (redeemed $10 discount)
SELECT
  COUNT(UNIQUE User.id) AS number_participant_users,
  COUNT(UNIQUE Purchase.id) AS number_recurring_orders,
  SUM(Purchase.total) AS total_revenue_existing_users,
  (CASE WHEN Purchase.date >= '2021-03-01'
  AND Purchase.date < '2021-05-01' THEN 'post_reward'
  WHEN Purchase.date >= '2021-01-01'
  AND Purchase.date < '2021-03-01' THEN 'pre_reward'
  ELSE NULL
  ) AS reward_program_period,
  (CASE WHEN Purchase.user_discounts = 10 THEN 'redemption_10'
   WHEN Purchase.user_discounts = 20 THEN 'redemption_20'
   WHEN Purchase.user_discounts = 0 THEN 'redemption_0'
   ELSE 'Other'
   ) AS amount_of_redemption
FROM
  User
  RIGHT OUTER JOIN Purchase ON User.id = Purchase.user_id
WHERE
  Purchase.total IS NOT NULL
GROUP BY
  reward_program_period
HAVING
  amount_of_redemption = 'redemption_10' 



--Goal2: To examine the difference between pre- and post- program in terms of
--      total number of new users and revenue from referral (redeemed $20 discount).
SELECT
  COUNT(UNIQUE User.id) AS number_new_users,
  COUNT(UNIQUE Purchase.id) AS number_new_orders,
  SUM(Purchase.total) AS total_revenue_new_users,
  (CASE WHEN Purchase.date >= '2021-03-01'
  AND Purchase.date < '2021-05-01' THEN 'post_reward'
  WHEN Purchase.date >= '2021-01-01'
  AND Purchase.date < '2021-03-01' THEN 'pre_reward'
  ELSE NULL
  ) AS reward_program_period,
  (CASE WHEN Purchase.user_discounts = 10 THEN 'redemption_10'
    WHEN Purchase.user_discounts = 20 THEN 'redemption_20'
    WHEN Purchase.user_discounts = 0 THEN 'redemption_0'
    ELSE 'Other'
  ) AS amount_of_redemption
FROM
  User
RIGHT OUTER JOIN 
  Purchase 
ON 
  User.id = Purchase.user_id
WHERE
  User.referring_user_id IS NOT NULL
GROUP BY
  reward_program_period
HAVING
  amount_of_redemption = 'redemption_10' 
  
  
  
--Goal3: To examine the difference between pre- and post- program in terms of
--      total revenue and total sales
SELECT
  COUNT(UNIQUE User.id) AS number_users,
  COUNT(UNIQUE Purchase.id) AS number_new_orders,
  SUM(Purchase.total) AS total_revenue_new_users,
  (CASE WHEN Purchase.date >= '2021-03-01'
  AND Purchase.date < '2021-05-01' THEN 'post_reward'
  WHEN Purchase.date >= '2021-01-01'
  AND Purchase.date < '2021-03-01' THEN 'pre_reward'
  ELSE NULL
  ) AS reward_program_period
FROM
  User
RIGHT OUTER JOIN 
  Purchase 
ON 
  User.id = Purchase.user_id
GROUP BY
  reward_program_period
'''

### 2) Using the information from Step 1), how would you make a recommendation on whether this rewards program should be continued or discontinued

**From the result from step 1), we will get:**
* Total number of users that redeemed \$10
* Total number of users that redeemed \$20
* Total number of orders with a redemption of \$10
* Total number of orders with a redemption of \$20
* Total revenue obtained in the 2 month prior to the program
* Total revenue obtained in the 2 month after program begins

**Insights:**
1. If *the total number of users that redeemed \$10* takes a vast majority (say 90\%+) of the existing users who order during the reward program period, then the participation of this campain can be gauranteed.
2. The proportion of *total number of users that redeemed \$20* divided by *total number of users that redeemed \$10* can evaluate the exact impact of referral message (e.g. conversion rate). A higher rate indicates larger impact, and will make analyst more willing to make a recommendation.
3. Since one unique user can order more than once and be qualified for \$10 discount more than once. The proportion of *total number of orders with a redemption of \$20* divided by *total number of orders with a redemption of \$10* can be treated as a validation and addition to the previous point.
4. Comparing data of the 2 month after the reward program begins with the data of 2 month prior to the campaign, if the total revenue increases significantly, we may conclude that this campain is effective in driving higher revenue; if the total number of orders increases significantly, we may conclude that this program works well in driving new orders. 
5. All above metrics should be considered combiningly while determining whether to make a recommendation or not.
6. All above steps should be taken again when we change control group from 01/01/2021-03/01/2021 to 03/01/2020-05/01/2020 as a validation.



### Coding Question
Goal: looking for a program that manages disjointed intervals of integers.

In [9]:
# First define the `add` function
intervals = []
def add(_from, to):
    '''
    This function is for managing disjointed intervals of integers-add an inverval
    '''
    newInterval=[_from, to]
    intervals.append(newInterval) # incorporate all intervals
    intervals.sort()
    res=[]
    res.append(intervals[0]) # assign the smallest interval to the output
    for i in intervals[1:]:
        if i[0]<=res[-1][1]:  # if two interval overlaps, merge
            res[-1][1]=max(res[-1][1],i[1])
        else:
            res.append(i)               
    return res

In [10]:
# Run a small test:
intervals=[[1, 2], [3, 5]]
add(6,8)

[[1, 2], [3, 5], [6, 8]]

In [12]:
# Then define a function to identify all overlaping cases
def cut(a1,a2,b1,b2):
    '''
    First define rules
    '''
    if a1 <= b1 and a2 < b2:  # left side remains after removal
        return [[a1, b1],]
    if b1 <= a1 and b2 < a2:  # right side remains after removal
        return [[b2, a2],]
    if a1 <= b1 and b2 < a2:  # mid side removed
         return [[a1, b1], [b2, a2]]
    return list()

# Define the removal function  
def remove(_from,to):
    result = []
    for i in intervals:
        lo, hi = i
        if lo <= to and _from <= hi:
            result.extend(cut(lo, hi, _from, to)) # call the `cut` function above
        else:
            result.append(i)
    return result

In [13]:
# Test on the `remove` function
intervals = [[1, 2], [3, 5], [6, 8]]
remove(4,7)

[[1, 2], [3, 4], [7, 8]]

#### Full Test

In [14]:
# True Answer:
# Start = []
# add(1, 5) => [[1, 5]]
# remove(2, 3) => [[1, 2], [3, 5]]
# add(6, 8) => [[1, 2], [3, 5], [6, 8]]
# remove(4, 7) => [[1, 2], [3, 4], [7, 8]]
# add(2, 7) => [[1, 8]]

# Test Result:
intervals = []
intervals = add(1, 5)
intervals = remove(2, 3)
intervals = add(6, 8)
intervals = remove(4, 7)
intervals = add(2, 7)
intervals

[[1, 8]]